In [1]:
# Plotly and Dash library
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objs as go
import plotly.express as px
import plotly

# Arrays and Dataframe
from collections import deque  # An array that can contain a certain amount of values
                               # it deletes automatically the oldest
import pandas as pd
import numpy as np

# Other
import random
import json
import ast

# Custom
from dashboard_utils import *

In [2]:
'''Gets the data from file and creates'''
def counttable_getdata():
    message = load_obj('message')
    df = pd.DataFrame()
    data = {'ch': 'All', 'counts': message['Total Count']}
    df = df.append(data, ignore_index=True)
    for i in range(4):
        data = {'ch': 'Chamber '+str(i), 'counts':message['Chamber_'+str(i+1)]['Count']}
        df = df.append(data, ignore_index=True)
    
    return df.to_dict('records')

def hist_getdata(numhist,chamber):
    message = load_obj('message')
    
    data = {'y': message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Counts'],
            'x': message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Bins'][1:],
            'type':'bar'}
    
    return {'data': [data], 'layout' : go.Layout(title='Chamber '+str(chamber))}

In [3]:
tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'padding': '6px',
    'fontWeight': 'bold'
}

hist_div_style = {
 'display': 'inline-block', 'width':'50%'
}

hist_style = {
 'height':'20vw'
}

In [ ]:
index = [0]

totalcount_X = deque(maxlen=10)
totalcount_X.append(0)
totalcount = deque(maxlen=10)
totalcount.append(0)

tblcols=[{'name': 'Chamber', 'id': 'ch'},
         {'name': 'Counts', 'id': 'counts'}]

app = dash.Dash(__name__)

# LAYOUT OF THE WEB PAGE
app.layout = html.Div(
    [   # TITLE
        html.H2('Live Dashboard 🔴'),
        # to make it side by side you need both elements to be in children=[,] and have style={'display': 'inline-block'}
        html.Div(children=[
            # Container of Graph
            html.Div(dcc.Graph(id='live-graph', animate=True),
                     style={'width': '60%', 'display': 'inline-block',"margin-right": "5%"}),
            # Container of Table
            html.Div(dash_table.DataTable(id = 'count_table', data = counttable_getdata(), columns=tblcols,
                                          style_header=
                                            {
                                                'fontWeight': 'default'
                                            },
                                          style_table={'width': '180%'},
                                          style_cell={'textAlign': 'left', 'fontSize': '120%'},
                                          style_cell_conditional=[
                                            {
                                                'if': {'column_id': 'ch'},
                                                'fontWeight': 'bold'
                                            }]),style={'display': 'inline-block',"verticalAlign": "top",
                                                       "margin-top": "5%"})
            
        ]),
        html.Div([
            dcc.Tabs(id='tabs', value='tab-1', children=[
                    dcc.Tab(label='Histogram 1', value='tab-1', style=tab_style, selected_style=tab_selected_style,
                            children=[html.Div(dcc.Graph(id = 'hist1-1', animate=True,
                                                figure = hist_getdata(1,1), 
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist1-2', animate=True,
                                                figure = hist_getdata(1,2), 
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist1-3', animate=True,
                                                figure = hist_getdata(1,3), 
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist1-4', animate=True,
                                                figure = hist_getdata(1,4), 
                                                style=hist_style),style=hist_div_style)
                                     ]),
                    dcc.Tab(label='Histogram 2', value='tab-2', style=tab_style, selected_style=tab_selected_style,
                            children=[html.Div(dcc.Graph(id = 'hist2-1', animate=True,
                                                figure = hist_getdata(2,1), 
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist2-2', animate=True,
                                                figure = hist_getdata(2,2), 
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist2-3', animate=True,
                                                figure = hist_getdata(2,3), 
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist2-4', animate=True,
                                                figure = hist_getdata(2,4), 
                                                style=hist_style),style=hist_div_style)
                            ]),
                ])]),
    dcc.Interval(id='graph-update')]
)

# CALLBACK FOR COUNT GRAPH
@app.callback(Output('live-graph', 'figure'),
        [Input('graph-update', 'n_intervals')])
def update_graph_scatter(n):
    message = load_obj('message')
    if message['Index'] != index[-1]:
        totalcount_X.append(totalcount_X[-1] + 1)
        totalcount.append(message['Total Count'])
        
        index.append(message['Index'])

    data = plotly.graph_objs.Scatter(
            x=list(totalcount_X),
            y=list(totalcount),
            name='Scatter',
            mode= 'lines+markers'
            )

    return {'data': [data],'layout' : go.Layout(height=300,
                                                title='Total Count of Events',
                                                xaxis=dict(range=[min(totalcount_X),max(totalcount_X)]),
                                                yaxis=dict(range=[min(totalcount),max(totalcount)]),)}

# CALLBACK FOR TABLE
@app.callback(Output('count_table','data'),
             [Input('graph-update', 'n_intervals')])
def updateTable(n):
     return counttable_getdata()
    
# CALLBACK FOR HIST
@app.callback(Output('hist1-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist1(n):
     return hist_getdata(1,1)
    
@app.callback(Output('hist1-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist2(n):
     return hist_getdata(1,2)
    
@app.callback(Output('hist1-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist3(n):
     return hist_getdata(1,3)
    
@app.callback(Output('hist1-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist4(n):
     return hist_getdata(1,4)
@app.callback(Output('hist2-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist5(n):
     return hist_getdata(2,1)
    
@app.callback(Output('hist2-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist6(n):
     return hist_getdata(2,2)
    
@app.callback(Output('hist2-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist7(n):
     return hist_getdata(2,3)
    
@app.callback(Output('hist2-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist8(n):
     return hist_getdata(2,4)
    
    
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
